# Policy iteration and value iteration

In this notebook, you will implement different dynamic programming approaches described in [Sutton and Barto's book, Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html). A grid ```World``` class and policy iteration has been implemented. Feel free to add more actions, rewards and/or terminals, or to modify the code to suit your needs.

### Install dependencies

In [207]:
! pip install numpy pandas

### Imports

In [208]:
import numpy as np
import sys          # We use sys to get the max value of a float
import pandas as pd # We only use pandas for displaying tables nicely
pd.options.display.float_format = '{:,.3f}'.format

### ```World``` class and globals

The ```World``` is a grid represented as a two-dimensional array of characters where each character can represent free space, an obstacle, or a terminal. Each non-obstacle cell is associated with a reward that an agent gets for moving to that cell (can be 0). The size of the world is _width_ $\times$ _height_ characters.

A _state_ is a tuple $(x,y)$.

An empty world is created in the ```__init__``` method. Obstacles, rewards and terminals can then be added with ```add_obstacle``` and ```add_reward```.

To calculate the next state of an agent (that is, an agent is in some state $s = (x,y)$ and performs and action, $a$), ```get_next_state()```should be called. It will only be relevant to call this function later on, when we do learning based on interaction with the environment and where an agent actually has to move.

For now, you will only need the probabilities over next states given an action, $a$, that is, call ```get_state_transition_probabilities```.

In [209]:
# Globals:
ACTIONS = ("up", "down", "left", "right")

# Rewards, terminals and obstacles are characters:
REWARDS = {" ": 0, ".": 0.1, "+": 10, "-": -10}
TERMINALS = ("+", "-") # Note a terminal should also have a reward assigned
OBSTACLES = ("#")

# Discount factor
gamma = 1

# The probability of a random move:
rand_move_probability = 0

class World:
  def __init__(self, width, height):
    self.width = width
    self.height = height
    # Create an empty world where the agent can move to all cells
    self.grid = np.full((width, height), ' ', dtype='U1')

  def add_obstacle(self, start_x, start_y, end_x=None, end_y=None):
    """
    Create an obstacle in either a single cell or rectangle.
    """
    if end_x == None: end_x = start_x
    if end_y == None: end_y = start_y

    self.grid[start_x:end_x + 1, start_y:end_y + 1] = OBSTACLES[0]

  def add_reward(self, x, y, reward):
    assert reward in REWARDS, f"{reward} not in {REWARDS}"
    self.grid[x, y] = reward

  def add_terminal(self, x, y, terminal):
    assert terminal in TERMINALS, f"{terminal} not in {TERMINALS}"
    self.grid[x, y] = terminal

  def is_obstacle(self, x, y):
    if x < 0 or x >= self.width or y < 0 or y >= self.height:
      return True
    else:
      return self.grid[x ,y] in OBSTACLES

  def is_terminal(self, x, y):
    return self.grid[x ,y] in TERMINALS

  def get_reward(self, x, y):
    """
    Return the reward associated with a given location
    """
    return REWARDS[self.grid[x, y]]

  def get_next_state(self, current_state, action, deterministic=False):
    """
    Get the next state given a current state and an action. Can eiter be
    deterministic (no random actions) or non-deterministic,
    where rand_move_probability determines the probability of ignoring the
    action and performing a random move.
    """
    assert action in ACTIONS, f"Unknown acion {action} must be one of {ACTIONS}"

    x, y = current_state

    # If our current state is a terminal, there is no next state
    if self.grid[x, y] in TERMINALS:
      return None

    # Check of a random action should be performed:
    if not deterministic and np.random.rand() < rand_move_probability:
      action = np.random.choice(ACTIONS)

    if action == "up":      y -= 1
    elif action == "down":  y += 1
    elif action == "left":  x -= 1
    elif action == "right": x += 1

    # If the next state is an obstacle, stay in the current state
    return (x, y) if not self.is_obstacle(x, y) else current_state

  def get_state_transition_probabilities(self, current_state, action):
    """
    Returns a dict where key = state and value = probability given current state
    is (x,y) and "action" is performed.
    """
    assert action in ACTIONS, f"Unknown acion {action} must be one of {ACTIONS}"

    x, y = current_state
    if self.is_terminal(x, y):
      return {}

    next_state_probabilities = {}
    # Since there is rand_move_probability of performing any action, we have to
    # go through all actions and check what their next state would be:
    for a in ACTIONS:
      next_state = self.get_next_state((x, y), a, deterministic=True)
      if a == action:
        prob = 1 - rand_move_probability + rand_move_probability / len(ACTIONS)
      else:
        prob = rand_move_probability / len(ACTIONS)

      if next_state in next_state_probabilities:
        next_state_probabilities[next_state] += prob
      else:
        if prob > 0.0:
          next_state_probabilities[next_state] = prob

    return next_state_probabilities

## Basic examples: World, obstacles, rewards and terminals

Below are some examples to illustrate how the ```World``` class works.

First, we create a 4x4 world:

In [210]:
world = World(4, 4)

# Note, that we have to transpose the 2D array (.T) for (x,y)
# to match the convention when displayed
print(world.grid.T)

[[' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']]


Obstacles and terminals are all represented as single characters:

In [211]:
print(f"Obstacles: {OBSTACLES}")
print(f"Terminals: {TERMINALS}")

Obstacles: #
Terminals: ('+', '-')


Rewards are also represented as characters in the world, but they have an associated value (note that defining a value for an empty space "  " is equivalent to the agent receiving that reward each time a move is made):

In [212]:
print(f"Rewards: {REWARDS}")

Rewards: {' ': 0, '.': 0.1, '+': 10, '-': -10}


To assign rewards to terminal states, just use the same character in the ```REWARDS``` dict and in the ```TERMINALS``` tuple.

In [213]:
for t in TERMINALS:
  print(f"Terminal {t} has reward {REWARDS[t]}")

world.add_terminal(0, 0, "+")
world.add_terminal(3, 3, "-")


print(world.grid.T)

# An alternative way of displaying the world using pandas:
display(pd.DataFrame(world.grid.T))

Terminal + has reward 10
Terminal - has reward -10
[['+' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' '-']]


,0,1,2,3
0,+,,,
1,,,,
2,,,,
3,,,,-


## Policies ($\pi$)

Recall that a policy, $\pi(a|s) = \Pr(A_t = a | S_t = s)$, maps states to action probabilities. In the code below, we let policies return the probabilities of each possible action given a state. States are $(x, y)$ coordinates and the policy must return action probabilities as a dict where the action is the ```key``` and the corresponding ```value``` is the probability of taking that action in the given state. Deterministic policies, for instance, return a dict with only one entry (e.g. ```{ "up": 1 } ``` if the action for the current state is ```up```).

A random policy can be defined as follows:


In [214]:
def equiprobable_random_policy(x, y):
  return { k : 1 / len(ACTIONS) for k in ACTIONS }

# Example (since the action probabilities do not depend on the state in this
# basic policy, we can just call it for state (0, 0)):
print(equiprobable_random_policy(0, 0))

{'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}


## Iterative policy evaluation


Iterative policy evaluation takes a ```World```, a discount factor, $\gamma$ (```gamma```, defined above in the ```World``` code cell), a policy, $\pi$, and a threshold, $\theta$ (```theta```), that determines when to stop the iteration. You can also specify a maximum number of iterations which can be useful for debugging using the ```max_iterations``` argument.

**IMPORTANT:** Remember that in iterative policy evaluation, we just learn state values ($V_\pi$) given a policy $\pi$. We are **not** trying to learn a policy.

(see page 74-75 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html) for an explanation and the algorithm)


In [215]:
def iterative_policy_evaluation(world, policy, theta=1e-5, max_iterations=1e3): #1e3

  # Our initial estimates for all states in the world is 0:
  V = np.full((world.width, world.height), 0.0)
  counter = 0
  while True:
    counter += 1
    # delta keeps track of the largest change in one iteration, so we set it to
    # 0 at the start of each iteration:
    delta = 0

    # Loop over all states (x,y)
    for y in range(world.height):
      for x in range(world.width):
        if not world.is_obstacle(x, y):
          # Get action probabilities for the current state:
          actions = policy(x, y)

          # v is the new estimate that will be updated in the loop:
          v = 0

          # loop over all actions that our policy says that we can perform
          # in the current state:
          for action, action_prob in actions.items():
            # For each action, get state transition probabilities and
            # accumulate in v rewards weighted with action and state transition
            # probabilities:
            for (xi, yi), state_prob in world.get_state_transition_probabilities((x, y), action).items():
              v += action_prob * state_prob * (world.get_reward(xi, yi) + gamma * V[xi, yi])

          # update delta (largest change in estimate so far)
          delta = max(delta, abs(v - V[x, y]))
          V[x, y] = v

    # check if current state value estimates are close enought to end:
    if delta <= theta:
      break

    max_iterations -= 1
    if max_iterations == 0:
      break

  print(counter)
  # Return the state value estimates
  return V


## Implementation of Example 4.1 from the book

Below, you can see the implementation of Example 4.1 on page 76 in the book [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html)

In [216]:
# World is 4x4
world = World(4, 4)

# Rewards are -1 for each move (including when hitting a terminal state, "+"):
REWARDS = {" ": -1, "+": -1}


# Add terminal states in two corners
world.add_terminal(0, 0, "+")
world.add_terminal(3, 3, "+")

print(world.grid.T)

[['+' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' '+']]


In [217]:
gamma = 1
V = iterative_policy_evaluation(world, equiprobable_random_policy)

display(pd.DataFrame(V.T))

141


,0,1,2,3
0,0.000,-14.000,-20.000,-22.000
1,-14.000,-18.000,-20.000,-20.000
2,-20.000,-20.000,-18.000,-14.000
3,-22.000,-20.000,-14.000,0.000


## Exercise - policy and discount factor

Experiment with example 4.1: what effect does it have to change the policy, e.g. so that an agent always goes left or always goes right? What effect does it have on state values to change the value of the discount factor (```gamma```)?

In [218]:
# TODO: implement your code here
def go_right_policy(x, y):
  return { "right": 1 }



V1 = iterative_policy_evaluation(world, go_right_policy)
print(f"go_right_policy | gamma: {gamma}")
display(pd.DataFrame(V1.T))
print("\n")
gamma = 0.9
V2 = iterative_policy_evaluation(world, equiprobable_random_policy)

print(f"equiprobable_random_policy | gamma: {gamma}")
display(pd.DataFrame(V2.T))

1000
go_right_policy | gamma: 1


,0,1,2,3
0,0.000,"-1,000.000","-1,000.000","-1,000.000"
1,"-1,000.000","-1,000.000","-1,000.000","-1,000.000"
2,"-1,000.000","-1,000.000","-1,000.000","-1,000.000"
3,-3.000,-2.000,-1.000,0.000




50
equiprobable_random_policy | gamma: 0.9


,0,1,2,3
0,0.000,-5.278,-7.128,-7.650
1,-5.278,-6.606,-7.181,-7.128
2,-7.128,-7.181,-6.606,-5.278
3,-7.650,-7.128,-5.278,0.000


### Så hvis man sætter gamma(discount rate) ned til 0.9 så bliver man mindre straffet for fremtidige actions => lavere probabilities for at nå til terminal state

Try to write a policy that is deterministic, but where the action performed differs between states. You can implement it as a two-dimensional array with the dimensions corresponding to the world dimensions and have each entry be an action for the corresponding state.

In [219]:
# TODO: implement you code here
def deterministic_policy(x, y):
  if x == 0:
    return { "up": 1 }
  else:
    return { "left": 1}
print(deterministic_policy(2, 2))

gamma = 1
V3 = iterative_policy_evaluation(world, deterministic_policy)
display(pd.DataFrame(V3.T))


{'left': 1}
2


,0,1,2,3
0,0.000,-1.000,-2.000,-3.000
1,-1.000,-2.000,-3.000,-4.000
2,-2.000,-3.000,-4.000,-5.000
3,-3.000,-4.000,-5.000,0.000


## Exercise - stochasticity

You can adjust the degree of stochasticity in the environment by setting the global variable ```rand_move_probability``` (the probability of the world ignoring an action and performing a random move instead). What effect does stochasticity have on the state-value estimates?


In [220]:
# TODO: implement you code here
rand_move_probability = 0.5

V4 = iterative_policy_evaluation(world, deterministic_policy)
display(pd.DataFrame(V4.T))

32


,0,1,2,3
0,0.000,-2.455,-4.737,-6.567
1,-2.462,-4.450,-6.317,-7.714
2,-4.781,-6.519,-7.837,-7.843
3,-6.644,-7.954,-7.944,0.000


### Du bliver straffet hårdere
### Her er det interessant at pga der tages random moves så er felterne nederst til højre pludselig lidt mere attraktive for de har chancen for at speedrunne til terminal state på (3,3)

## Exercise - robot, cake and mouse trap

Implement a robot, cake and mouse trap example and compute state value estimates under different policies (equiprobable, always right, always right:50% or up:50%) with and without stochasticity.


In [221]:
# TODO: implement you code here
# cake_trap_world is 4x4
cake_trap_world = World(4, 4)

# Rewards are -1 for each move (including when hitting a terminal state, "+"):
REWARDS = {" ": -1, "+": -1, "C": 15, "T": -10}


# Add terminal states in two corners
cake_trap_world.add_terminal(0, 0, "+")
cake_trap_world.add_terminal(3, 3, "+")

cake_trap_world.add_reward(2,0, "C")
cake_trap_world.add_reward(2,1, "T")


print(cake_trap_world.grid.T)

rand_move_probability = 0

V5 = iterative_policy_evaluation(cake_trap_world, equiprobable_random_policy)
display(pd.DataFrame(V5.T))
V6 = iterative_policy_evaluation(cake_trap_world, go_right_policy)
display(pd.DataFrame(V6.T))


[['+' ' ' 'C' ' ']
 [' ' ' ' 'T' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' '+']]
133


,0,1,2,3
0,0.000,0.747,0.511,2.357
1,-9.824,-10.269,-4.571,-7.797
2,-15.203,-14.428,-12.731,-8.176
3,-17.357,-15.511,-10.747,0.000


1000


,0,1,2,3
0,0.000,-984.000,"-1,000.000","-1,000.000"
1,"-1,009.000","-1,009.000","-1,000.000","-1,000.000"
2,"-1,000.000","-1,000.000","-1,000.000","-1,000.000"
3,-3.000,-2.000,-1.000,0.000


In [222]:
def right_up_policy(x, y):
    return { "up": 0.5, "right": 0.5 }
  
V7 = iterative_policy_evaluation(cake_trap_world, right_up_policy)
display(pd.DataFrame(V7.T))

1000


,0,1,2,3
0,0.000,-968.000,-984.000,"-1,000.000"
1,-491.250,-981.500,-985.000,"-1,001.000"
2,-741.375,-990.500,-998.500,"-1,002.000"
3,-744.375,-746.125,-500.250,0.000


### Med stochastisitet

In [223]:
rand_move_probability = 0.5

V8 = iterative_policy_evaluation(cake_trap_world, equiprobable_random_policy)
display(pd.DataFrame(V8.T))
V9 = iterative_policy_evaluation(cake_trap_world, go_right_policy)
display(pd.DataFrame(V9.T))
V10 = iterative_policy_evaluation(cake_trap_world, right_up_policy)
display(pd.DataFrame(V10.T))


133


,0,1,2,3
0,0.000,0.747,0.511,2.357
1,-9.824,-10.269,-4.571,-7.797
2,-15.203,-14.428,-12.731,-8.176
3,-17.357,-15.511,-10.747,0.000


336


,0,1,2,3
0,0.000,-2.005,-12.466,-13.022
1,-20.750,-23.701,-19.147,-21.579
2,-18.744,-18.118,-17.119,-15.566
3,-11.873,-8.898,-4.860,0.000


744


,0,1,2,3
0,0.000,78.156,93.850,91.132
1,35.019,69.228,86.702,80.413
2,45.447,59.235,61.597,58.967
3,43.367,43.954,31.535,0.000


## Exercise - action-value function

Based on a set of calculated state values, try to implement an action value function, that is $q_\pi(s, a)$ (if in doubt, see page 78 in [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html)). Note: you have to use the ```get_state_transition_probabilities()``` method on ```World``` to be able to handle stochastic environments where performing ```a``` does not lead to a deterministic outcome.

In [224]:
ct_world = World(4, 4)

# Rewards are -1 for each move (including when hitting a terminal state, "+"):
REWARDS = {" ": -1, "+": 15, "-": -10}


# Add terminal states in two corners
ct_world.add_terminal(3, 0, "+")
ct_world.add_terminal(3, 1, "-")
ct_world.add_obstacle(1,2)

V11 = iterative_policy_evaluation(ct_world, go_right_policy)
display(pd.DataFrame(V11.T))

218


,0,1,2,3
0,0.422,4.413,9.521,0.000
1,-11.537,-9.135,-9.764,0.000
2,-27.505,0.000,-25.499,-25.373
3,-35.474,-35.468,-33.867,-33.620


In [225]:


def action_value(world, V, state, action):
  # TODO: implement your code here
  q = 0
  for s_nxt, prob_nxt in world.get_state_transition_probabilities(state, action).items():
    reward = world.get_reward(s_nxt[0],s_nxt[1])
    # print(reward)
    q += prob_nxt * (reward + gamma * V[s_nxt])
  return q

rand_move_probability = 0
gamma = 1
print(action_value(ct_world, V11, (1,1), "up"))




3.4133317859458963


## Exercise - policy iteration

You are now ready to implement policy iteration. That is, first estimate state values under a given policy, then improve the policy based on those estimates and action values, estimate state values again, and so on. See page 80 in [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html)

You will need an explicit representation of your policy that you can easily change.

Test your implementation and print out the policies found.


In [226]:
go_right_pol = np.full((world.width, world.height), "right")

def explicit_policy(x,y):
    return { go_right_pol[x,y]: 1 }

print(go_right_pol[(2,2)])

right


In [227]:
# TODO: Implement your code here
def policy_iteration(world, policy):
    # 1 Initialization and 2 Policy evaluation from given function
    my_pol = policy
    
    while True:
        V = iterative_policy_evaluation(world, explicit_policy)

        policy_stable = True
    
        for y in range(world.height):
            for x in range(world.width):
                if not world.is_obstacle(x, y):
                    old_action = my_pol[(x,y)]
                    best_act = action_value(world, V, (x,y), my_pol[(x,y)])
                    for act in ["up", "right", "down", "left"]:   
                        curr_act = action_value(world, V, (x,y), act)
                        if curr_act > best_act:
                            best_act = curr_act
                            my_pol[(x,y)] = act
                    if old_action != my_pol[(x,y)]:
                        policy_stable = False
        if policy_stable:
            break
            
    return V, my_pol

In [228]:
V12, pol = policy_iteration(ct_world, go_right_pol)
display(pd.DataFrame(V12.T))
display(pd.DataFrame(pol.T))

1000
1000
4


,0,1,2,3
0,13.000,14.000,15.000,0.000
1,12.000,13.000,14.000,0.000
2,11.000,0.000,13.000,12.000
3,10.000,11.000,12.000,11.000


,0,1,2,3
0,right,right,right,right
1,up,up,up,right
2,up,right,up,left
3,up,right,up,up


## Exercise - value iteration

Value iteration is much more effecient than policy iteration. Implement value iteration below. See page 83 in [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html).

Test your implementation and display the policies found (i.e., a grid with the perferred action in each cell).

In [229]:
# TODO: Implement your code here

def value_iteration(world, theta=1e-5):
    # Our initial estimates for all states in the world is 0:
    V = np.full((world.width, world.height), 0.0)
    actions = ["up", "right", "down", "left"]
    counter = 0

    while True:
        delta = 0
        counter += 1
        for y in range(world.height):
            for x in range(world.width):
                if not world.is_obstacle(x, y):
                    v = V[x][y]
                    V[x][y] = max(action_value(world, V, (x, y), act) for act in actions) # Beregn den bedste værdi af handlinger og opdater V(x, y)
                    delta = max(delta, v - V[x][y])

        if delta < theta:
            break
    
    #deterministic policy
    policy = np.full((world.width, world.height), None)
    for y in range(world.height):
        for x in range(world.width):
            if not world.is_obstacle(x, y):
                best_action = max(actions, key=lambda act: action_value(world, V, (x,y), act))

                policy[x][y] = best_action

    return V, policy, counter

V13, value_policy, counter = value_iteration(ct_world)
print(counter)
display(pd.DataFrame(V13.T))
display(pd.DataFrame(value_policy.T))
display(pd.DataFrame(ct_world.grid.T))


3


,0,1,2,3
0,13.000,14.000,15.000,0.000
1,12.000,13.000,14.000,0.000
2,11.000,0.000,13.000,12.000
3,10.000,11.000,12.000,11.000


,0,1,2,3
0,right,right,right,up
1,up,up,up,up
2,up,None,up,left
3,up,right,up,up


,0,1,2,3
0,,,,+
1,,,,-
2,,#,,
3,,,,
